# Phoenix Biodiversity Intactness Index

### Author: Tom Gibbens-Matsuyama 
[GitHub Repo](https://github.com/tommats00/phoenix-biodiversity-index)

## About

### Purpose
Maricopa County has had the highest increase in developement since 2001 in the U.S. This rapid developement can have detrimental affects on the nearby ecosystems. The purpose of this repository is to analyze Biodiversity Intactness Index (BII) data from the [Microsoft Planetary Computer STAC catalog](https://planetarycomputer.microsoft.com/dataset/io-biodiversity) to assess the effects of this urbanization. Data will be compared and analyzed to discuss the difference in BII from 2017 to 2020. 